# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

### Please enter your own key in the "api_keys.py" file to run the code.

In [1]:
# Dependencies and Setup
#!pip install gmaps
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
city_weather_csv_file = "../WeatherPy/output_data/cities.csv"

city_weather_df = pd.read_csv(city_weather_csv_file)

city_weather_df.head()
city_weather_df.count()

City          550
Cloudiness    550
Country       547
Date          550
Humidity      550
Latitude      550
Longitude     550
Max Temp      550
Wind Speed    550
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure maps with unique API key
gmaps.configure(api_key=g_key)

In [25]:
# Store 'Latitude' and 'Longitude' into  locations. 
locations = city_weather_df[["Latitude", "Longitude"]].astype(float)

# Convert Humidity to float and store
# Also, handle NaN values
city_weather_df = city_weather_df.dropna()
humidity = city_weather_df["Humidity"].astype(float)
#locations,humidity

(     Latitude  Longitude
 0     33.0833   -16.3333
 1    -46.1927   168.8643
 2    -42.8794   147.3294
 3     39.5696     3.2096
 4     -8.8978   -35.1478
 ..        ...        ...
 545   21.3469  -158.0858
 546   13.4667    22.2000
 547   20.4500  -101.5167
 548   30.1897   -82.6393
 549   70.9963    24.6622
 
 [547 rows x 2 columns],
 0      63.0
 1      94.0
 2      83.0
 3      40.0
 4      63.0
        ... 
 545    82.0
 546    17.0
 547    20.0
 548    94.0
 549    80.0
 Name: Humidity, Length: 547, dtype: float64)

In [26]:
# Create a humidity Heatmap layer
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

#fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(city_weather_df["Humidity"]),
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig



Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
# Narrow down the DataFrame to find your ideal weather condition.
ideal_weather_df = city_weather_df

# A max temperature lower than 80 degrees but higher than 70.
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Max Temp"] < 80) & (ideal_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Cloudiness"] == 0]

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.
ideal_weather_df = ideal_weather_df.dropna()

ideal_weather_df


,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
41,Mar del Plata,0,AR,1612542159,60,-38.0023,-57.5575,73.40,8.05
81,Hermanus,0,ZA,1612542587,71,-34.4187,19.2345,75.99,5.01
145,Santiago,0,CL,1612542021,56,-33.4569,-70.6483,75.00,5.75
163,Castro,0,BR,1612542430,55,-24.7911,-50.0119,74.12,8.14
196,Kruisfontein,0,ZA,1612542435,63,-34.0033,24.7314,78.01,7.00
226,Grand Gaube,0,MU,1612542439,78,-20.0064,57.6608,79.00,4.00
252,Cap Malheureux,0,MU,1612542302,78,-19.9842,57.6142,79.00,4.00
348,North Port,0,US,1612542086,54,27.0442,-82.2359,72.00,5.99
366,Wichian Buri,0,TH,1612542320,42,15.6578,101.1060,77.90,6.04
418,Lima,0,PE,1612542131,69,-12.0432,-77.0282,75.20,5.75


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_df = ideal_weather_df

# create a params dict that will be updated with each iteration
params = {
    "key": g_key,
    "radius": 5000,
    "types": "lodging",
}

In [14]:
for index, row in hotel_df.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # get lat, lng from df
    latitude = row["Latitude"]
    longitude = row["Longitude"]

    # update location key value
    params["location"] = f"{latitude},{longitude}"
    
    # make request
    response = requests.get(base_url, params=params).json()
    #print(json.dumps(response, indent=4, sort_keys=True))
    
    try:
        #print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
41,Mar del Plata,0,AR,1612542159,60,-38.0023,-57.5575,73.40,8.05,Gran Hotel Mar del Plata
81,Hermanus,0,ZA,1612542587,71,-34.4187,19.2345,75.99,5.01,Misty Waves Boutique Hotel
145,Santiago,0,CL,1612542021,56,-33.4569,-70.6483,75.00,5.75,Sheraton Santiago Hotel and Convention Center
163,Castro,0,BR,1612542430,55,-24.7911,-50.0119,74.12,8.14,CHACARA BAILLY
196,Kruisfontein,0,ZA,1612542435,63,-34.0033,24.7314,78.01,7.00,Oyster Bay House Rental


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [23]:
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))


# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…